# Portfolio Optimizer

## This notebook walks through the steps needed to create the payload used by the Portfolio Optimization service

This notebook requires a tradable universe. The experimental version of the Portfolio Optimization service has a static data set for users to play with; a dynamic data set will be integrated in a future release. Note that much of the data in this sample set is _representative_ of actual data. 

Live, accurate data will be available upon the Beta release of the service. For now, the spreadsheet "Fidelity Universe Data_FINAL" contains all relevant information on the current sample data set.

## Step 1 - Load in Data Set Details

Load the file into DSX. Apparently there's a command that simply inserts the code into your notebook that converts the csv file to a pandas dataframe!

In [ ]:
from io import StringIO
import requests
import json
import pandas as pd

##### ENTER DSX OBJECT STORE CREDENTIALS #####
dsx_id = ""
dsx_pw = ""

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_c0ad3ca445274f9d94984cea37d66ebd(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_f8ee0f52dadee561e6fc64bc7a725c6edeb84230','domain': {'id': dsx_id},
            'password': dsx_pw}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

universe_data = pd.read_csv(get_object_storage_file_with_credentials_c0ad3ca445274f9d94984cea37d66ebd('MarketDataExperience', 'Fidelity Universe Data_FINAL.csv'))
universe_data.head()


Create the Expected Returns and Standard Deviation file:

# Step 2 - Inputs: Holdings, Benchmarks, Objectives, and Preferences

As with any optimization problem, there are a few things we need to determine before we get started:
- **Holdings**: Where are we starting from? A blank slate? Or does the user have existing holdings?
- **Benchmark**: If we're matching the properties of something, what are we matching?
- **Objective**: What are we trying to do? Are we minimizing or maximizing some property? Or are we matching a target?
- **Preferences**: What are the users unique preferences and constraints that we need to take into account when performing this calculation? Is there anything that can't be included? Do we need to have a specific weight at the end of our calculation?

In [ ]:
import json

#Initiliaze the request
optimization = {
    "portfolios": [],
    "objectives": [],
    "constraints": []
}

## Holdings

First we need to decide if we're starting from scratch, or if the user already has a portfolio of assets. We only need to specify the ID of the asset (our data set that we loaded above uses ticker) and the quantity in # of units. 

We also need to define how much cash - in addition to the assets in the portfolio - the client wants to add to this analysis.

We define this as an array of json objects:

In [3]:
##### PUT IN YOUR HOLDINGS HERE #####
Holdings = []
Holdings.append({"asset":"FDRXX","quantity":32000}) #Example of a holding

#Example cash infusion
Cash = 10000
#####################################

#Initilialize the 'tradeable universe' - note that position units should be populated where assets are held.
tradeable_universe = {
    "name": "Universe",
    "type":"root",
    "holdings":[]
}
for index, row in universe_data.iterrows():
    holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
    if holding:
        tradeable_universe["holdings"].append(h)
    else:
        tradeable_universe["holdings"].append({"asset":row["Ticker"],"quantity":0})

optimization["portfolios"].append(tradeable_universe)
# Debug
#print json.dumps(optimization, indent=4, sort_keys=True)
#print("Holdings: " + str(Holdings))
#print("Cash: " + str(Cash))

## Benchmark

Next we need to pick a benchmark for our analysis. If we're simply optimizing to minimize variance/maximize return objectively, then this step can be skipped.

Instead, in our analysis we assume that our advisor is recommending an "off-the-shelf" portfolio that meets our risk and return objectives that we want to use, with some caveats. Here we select that benchmark by assigning the name to the Benchmark variable below.

Choices are: 
- Conservative (income-focused)
- Moderate
- Balanced
- Growth
- Aggressive Growth
- Most Aggressive Growth

Benchmark amount sets the total amount that the benchmark should add up to. This requirement may be depricated soon...

In [4]:
#ENTER DETAILS HERE
Benchmark_name = "Aggressive Growth"
Benchmark_amount = 50000 #Is this necessary?

#Assemble the benchmark portfolio using the above details:
Benchmark_col_name = "Benchmark - " + str(Benchmark_name)
benchmark = {
    "name": Benchmark_name,
    "type": "benchmark",
    "holdings": []
}
Benchmark_holdings = universe_data.filter(items=["Ticker","Last Close Price",Benchmark_col_name])[universe_data[Benchmark_col_name] <> 0]
for index,row in Benchmark_holdings.iterrows():
    #Determine weight of each asset
    shares = Benchmark_amount * row[Benchmark_col_name] / row["Last Close Price"]
    holding = {"asset":row["Ticker"],"quantity":shares}
    benchmark["holdings"].append(holding)
    
optimization["portfolios"].append(benchmark)
#Debug
#print(Benchmark_holdings)
print(benchmark)

{'holdings': [{'asset': 'FLVEX', 'quantity': 601.4434643143544}, {'asset': 'FVDFX', 'quantity': 179.0830945558739}, {'asset': 'FGRTX', 'quantity': 397.8779840848806}, {'asset': 'FSLEX', 'quantity': 199.2825827022718}, {'asset': 'FTHRX', 'quantity': 687.442713107241}, {'asset': 'FNCMX', 'quantity': 88.73639375295788}, {'asset': 'FSIIX', 'quantity': 242.89531212047606}], 'type': 'benchmark', 'name': 'Aggressive Growth'}


## Objective

Now we need to describe the overall mission of this analysis. We're going to hard-code this to be a minimization of variance of the returns of the portfolio - the standard [Markowitz approach](https://www.math.ust.hk/~maykwok/courses/ma362/07F/markowitz_JF.pdf).

For this type of analysis, the only parameters we need to be concerned with are:
- Whether or not we're trying to match the variance of the benchmark, or simply minimizing it objectively.
- The time step, which is contingent on the data set (in our case it's 30 days)

In [5]:
optimization["objectives"] = [{
       "sense": "minimize",
       "measure": "variance",
       "attribute": "return",
       "portfolio": "Universe",
       "TargetPortfolio": Benchmark_name,
       "timestep": 30,
       "description": "minimize tracking error squared (variance of the difference between Universe portfolio and Benchmark returns) at time 30 days"
}]

#Debug 
print(optimization["objectives"])

[{'sense': 'minimize', 'description': 'minimize tracking error squared (variance of the difference between Universe portfolio and Benchmark returns) at time 30 days', 'timestep': 30, 'TargetPortfolio': 'Aggressive Growth', 'measure': 'variance', 'attribute': 'return', 'portfolio': 'Universe'}]


## Preferences

Now we get to the real meat of the analysis. This is where we can specify our custom constraints that will make this a _personalized_ analysis, resulting a portfolio with [as close to] identical properties as the benchmark portfolio, but with assets that meet the criteria specified below.

There's a lot of flexibility in how one would specify these constraints, but for the sake of this walkthrough, we're identifying three main types:

1. __Filtering__: Ensuring that a certain type of investment is not present in the portfolio (e.g. "sin stocks", military investments)
2. __ESG and Socially Responsible Investing Requirements__: Determine a minimum threshold for types of companies one would want included in a portfolio (e.g. I want my portfolio's average Governance score to be "High")
3. __Investment Allocation__: Determine specific weight requirements of the portfolio (e.g. 70% bonds, 30% stocks)

The user can also specify standard result requirements including:
- Allow/Prevent short-selling (e.g. weights can or can't go negative)
- Maximum allocation to any one security

In [6]:
##### CONSTRAINTS #####
#Uncomment the constraints you wish to apply:

#Filtering Constraints - "The final portfolio..."
Filtering_Constraints = []
#Filtering_Constraints.append("Has Tobacco")
#Filtering_Constraints.append("Has Alcohol")
#Filtering_Constraints.append("Has Gambling")
Filtering_Constraints.append("Has Military") #Ex) No military investments desired in the portfolio
#Filtering_Constraints.append("Has Fossil Fuels")
    
#ESG Constraints - Define both a field and an mean score desired (e.g. "Low","Average","High")
ESG_Constraints = []
#ESG_Constraints.append({"field":"Controversy","mean_score":"High"})
ESG_Constraints.append({"field":"Environmental","mean_score":"High"})
#ESG_Constraints.append({"field":"Governance","mean_score":"High"})
#ESG_Constraints.append({"field":"Social","mean_score":"High"})
#ESG_Constraints.append({"field":"Sustainability","mean_score":"High"})
                       
#Investment Allocation Constraints - I want to allocate [allocation]% of my portfoilio to be [value] identified by the [field] of the assets.
#inequality can be "greater-or-equal","less-or-equal" or "equal"
Allocation_Constraints = []
Allocation_Constraints.append({"field":"ActiveOrPassive","value":"Passive","allocation":.1,"inequality":"greater-or-equal"}) #Field values are "Active" or "Passive"
#Allocation_Constraints.append({"field":"Geography","value":"International","allocation":.1,"inequality":"less-or-equal"}) #Field values are "Domestic" or "International"
#Allocation_Constraints.append({"field":"Asset Class","value":"Equity","allocation":.1,"inequality":"equal"}) #Field values are "Equity","Corporate Bonds","Municipal Bonds","Mortages","Mixed","Money Market", or "Commodities"
#Allocation_Constraints.append({"field":"Sector","value":"Information Technology","allocation":.1,"inequality":"greater-or-equal"}) #Field values are..
#"Consumer Discretionary","Consumer Staples","Energy","Financials","Health Care","Industrials","Information Technology","Materials","Real Estate","Utilities","Commodity","Government", or "Diversified"

#Result requirements
AllowShortSales = False           #No short-selling
MaximumInvestmentWeight = .2      #20%
#MaximumNumberofPositions = 5     #Cardinality constraint on the portfolio
#MinimumNumberofPositions = 5     #Cardinality constraint on the portfolio

#Debug
print("Filtering Constraints: " + str(Filtering_Constraints))
print("ESG Constraints: " + str(ESG_Constraints))
print("Allocation Constraints: " + str(Allocation_Constraints))

Filtering Constraints: ['Has Military']
ESG Constraints: [{'field': 'Environmental', 'mean_score': 'High'}]
Allocation Constraints: [{'allocation': 0.1, 'field': 'ActiveOrPassive', 'inequality': 'greater-or-equal', 'value': 'Passive'}]


In [7]:
#FILTERING CONTRAINTS
#Add sub-portfolio (how the optimization algorithm knows which asset has which property)
for f in Filtering_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name":f,
        "type":"subportfolio",
        "holdings":[]
    }
    
    #find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",f])[universe_data[f] <> 0]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(h)
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":f,
        "InPortfolio":"Universe",
        "relation":"equal",
        "constant":0.0,
        "description":"Excluding all securities which have the property " + f + "."
    })
    
#Debug
print json.dumps(subportfolio, indent=4, sort_keys=True)

{
    "ParentPortfolio": "Universe", 
    "holdings": [
        {
            "asset": "FLVEX", 
            "quantity": 0
        }, 
        {
            "asset": "FVDFX", 
            "quantity": 0
        }, 
        {
            "asset": "FSMVX", 
            "quantity": 0
        }, 
        {
            "asset": "FDVLX", 
            "quantity": 0
        }, 
        {
            "asset": "FCPVX", 
            "quantity": 0
        }, 
        {
            "asset": "FGRTX", 
            "quantity": 0
        }, 
        {
            "asset": "FMEIX", 
            "quantity": 0
        }, 
        {
            "asset": "FDCAX", 
            "quantity": 0
        }, 
        {
            "asset": "FCNTX", 
            "quantity": 0
        }, 
        {
            "asset": "FDFFX", 
            "quantity": 0
        }, 
        {
            "asset": "FMILX", 
            "quantity": 0
        }, 
        {
            "asset": "FBGRX", 
            "quantity": 0
        

In [8]:
#ESG CONSTRAINTS
#Add sub-portfolio (how the optimization algorithm knows which asset has which property)
for e in ESG_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name": e["mean_score"] + e["field"],
        "type":"subportfolio",
        "holdings":[]
    }
    
    #find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",e["field"]])[universe_data[e["field"]] == e["mean_score"]]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(h)
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":e["mean_score"] + e["field"],
        "InPortfolio":"Universe",
        "relation":"greater-or-equal",
        "constant":.5,
        "description":"Creating an average " + e["field"] + " score of " + e["mean_score"] + "."
    })
    
#Debug
print json.dumps(subportfolio, indent=4, sort_keys=True)

{
    "ParentPortfolio": "Universe", 
    "holdings": [
        {
            "asset": "FBCVX", 
            "quantity": 0
        }, 
        {
            "asset": "FVDFX", 
            "quantity": 0
        }, 
        {
            "asset": "FLPSX", 
            "quantity": 0
        }, 
        {
            "asset": "FSMVX", 
            "quantity": 0
        }, 
        {
            "asset": "FCPVX", 
            "quantity": 0
        }, 
        {
            "asset": "FEQTX", 
            "quantity": 0
        }, 
        {
            "asset": "FLCEX", 
            "quantity": 0
        }, 
        {
            "asset": "FDGRX", 
            "quantity": 0
        }, 
        {
            "asset": "FOCPX", 
            "quantity": 0
        }, 
        {
            "asset": "FDEGX", 
            "quantity": 0
        }, 
        {
            "asset": "FIENX", 
            "quantity": 0
        }, 
        {
            "asset": "FIVFX", 
            "quantity": 0
        

In [9]:
#ALLOCATION CONSTRAINTS
#Add sub-portfolio (how the optimization algorithm knows which asset has which property)
for a in Allocation_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name": a["value"],
        "type":"subportfolio",
        "holdings":[]
    }
    
    #find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",a["field"]])[universe_data[a["field"]] == a["value"]]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(h)
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":a["value"],
        "InPortfolio":"Universe",
        "relation":a["inequality"],
        "constant":a["allocation"],
        "description":"Weight of " + a["value"] + " in the portfolio should be " + a["inequality"] + " to " + str(a["allocation"]) + "."
    })
    
#Debug
print json.dumps(subportfolio, indent=4, sort_keys=True)

{
    "ParentPortfolio": "Universe", 
    "holdings": [
        {
            "asset": "FSCKX", 
            "quantity": 0
        }, 
        {
            "asset": "FENSX", 
            "quantity": 0
        }, 
        {
            "asset": "FNCMX", 
            "quantity": 0
        }, 
        {
            "asset": "FSIIX", 
            "quantity": 0
        }, 
        {
            "asset": "FSGUX", 
            "quantity": 0
        }, 
        {
            "asset": "FPEMX", 
            "quantity": 0
        }, 
        {
            "asset": "FPMAX", 
            "quantity": 0
        }, 
        {
            "asset": "FFNOX", 
            "quantity": 0
        }, 
        {
            "asset": "FBIDX", 
            "quantity": 0
        }, 
        {
            "asset": "FSBIX", 
            "quantity": 0
        }, 
        {
            "asset": "FSBAX", 
            "quantity": 0
        }, 
        {
            "asset": "FIBIX", 
            "quantity": 0
        

In [10]:
#RESULT REQUIREMENTS

#Short Sale Restriction
try:
    if AllowShortSales == False:
        optimization["constraints"].append({
           "attribute":"weight",
           "relation":"greater-or-equal",
           "members":"Universe",
           "constant":0,
           "description":"no short-sales for assets in Universe portfolio" 
        })
except:
    print("Short sales allowed")

#Maximum individual investment weight
try:
    optimization["constraints"].append({
       "attribute":"weight",
       "relation":"less-or-equal",
       "members":"Universe",
       "constant":MaximumInvestmentWeight,
       "description":"Weight of each asset from the Universe portfolio does not exceed " + str(MaximumInvestmentWeight*100) + "%."
    })
except:
    print("No maximum investment weight.")

#Maximum number of trades/positions
try:
    optimization["constraints"].append({
        "attribute": "count", 
        "relation": "less-or-equal", 
        "constant": MaximumNumberofPositions })
except:
    print("No maximum number of positions.")

#Minimum number of trades/positions
try:
    optimization["constraints"].append({
        "attribute": "count", 
        "relation": "greater-or-equal", 
        "constant": MinimumNumberofPositions })
except:
    print("No minimum number of positions.")

#Cash infusions
try:
    optimization["constraints"].append({
        "attribute:": "value",
        "portfolio": "Universe",
        "cashadjust": Cash,
        "description": "cash inflow of " + str(Cash) +" monetary units to the Universe portfolio"})
except:
    print("No cash infusions.")
    
# Debug
print json.dumps(optimization["constraints"], indent=4, sort_keys=True)

No maximum number of positions.
No minimum number of positions.
[
    {
        "InPortfolio": "Universe", 
        "attribute": "weight", 
        "constant": 0.0, 
        "description": "Excluding all securities which have the property Has Military.", 
        "portfolio": "Has Military", 
        "relation": "equal"
    }, 
    {
        "InPortfolio": "Universe", 
        "attribute": "weight", 
        "constant": 0.5, 
        "description": "Creating an average Environmental score of High.", 
        "portfolio": "HighEnvironmental", 
        "relation": "greater-or-equal"
    }, 
    {
        "InPortfolio": "Universe", 
        "attribute": "weight", 
        "constant": 0.1, 
        "description": "Weight of Passive in the portfolio should be greater-or-equal to 0.1.", 
        "portfolio": "Passive", 
        "relation": "greater-or-equal"
    }, 
    {
        "attribute": "weight", 
        "constant": 0, 
        "description": "no short-sales for assets in Universe por

## The Assembled Request:

We've assembled our payload! Below is what it looks like:

In [11]:
print json.dumps(optimization, indent=4, sort_keys=True)

{
    "constraints": [
        {
            "InPortfolio": "Universe", 
            "attribute": "weight", 
            "constant": 0.0, 
            "description": "Excluding all securities which have the property Has Military.", 
            "portfolio": "Has Military", 
            "relation": "equal"
        }, 
        {
            "InPortfolio": "Universe", 
            "attribute": "weight", 
            "constant": 0.5, 
            "description": "Creating an average Environmental score of High.", 
            "portfolio": "HighEnvironmental", 
            "relation": "greater-or-equal"
        }, 
        {
            "InPortfolio": "Universe", 
            "attribute": "weight", 
            "constant": 0.1, 
            "description": "Weight of Passive in the portfolio should be greater-or-equal to 0.1.", 
            "portfolio": "Passive", 
            "relation": "greater-or-equal"
        }, 
        {
            "attribute": "weight", 
            "constant": 0, 


# Step 3 - Submit the Request

Now that we have the payload assembled, we issue a [requests](http://docs.python-requests.org/en/master/) command to the Portfolio Optimization service running on the cloud. You'll want to provision your own copy of the application [which can be done here on Bluemix](https://console.bluemix.net/catalog/services/portfolio-optimization?env_id=ibm:yp:us-south). Fill in your uri and API credentials below before issuing the command.

In [12]:
##### ENTER YOUR PORTFOLIO OPTIMZIATION SERVICE CREDENTIALS #####
uri = "fss-analytics.mybluemix.net"
api_key = ""
#################################################################

url = "https://" + uri + "/api/v1/optimization/portfolio/construct"
headers = {'content-type': 'application/json', 'accept': 'application/json', 'X-IBM-Access-Token': api_key}
data = json.dumps(optimization)

import requests
r = requests.post(url,data=data,headers=headers)

print(r.text)


{
 "Holdings": [
  {
   "Asset": "FBCVX",
   "Quantity": 0,
   "OptimizedTrade": 8.52802862476909,
   "OptimizedQuantity": 8.52802862476909
  },
  {
   "Asset": "FLVEX",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "FVDFX",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "FLPSX",
   "Quantity": 0,
   "OptimizedTrade": 4.19366728603124,
   "OptimizedQuantity": 4.19366728603124
  },
  {
   "Asset": "FSMVX",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "FDVLX",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "FCPVX",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "FEQTX",
   "Quantity": 0,
   "OptimizedTrade": 1.18725515567421,
   "OptimizedQuantity": 1.18725515567421
  },
  {
   "Asset": "FEQIX",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Ass

# Step 4 - Viewing your Results

So let's take a look at the results of our optimization calculation:

In [13]:
import pixiedust
pixiedust.installPackage("graphframes:graphframes:0.1.0-spark1.6")

#Initialize Portfolio and field names
new_portfolio = []
fields = ["Name","Last Close Price"]

#Gather fields
for f in Filtering_Constraints:
    fields.append(f)
for e in ESG_Constraints:
    fields.append(e["field"])
#for a in Allocation_Constraints:
 #   fields.append(a["field"])
    
#Assemble the data frame
for i in json.loads(r.text)["Holdings"]:
    if i["OptimizedQuantity"] != 0:
        security_data = universe_data.filter(items=fields)[universe_data["Ticker"] == i["Asset"]]
        security_data = security_data.values.tolist()[0]
        security_data.append(i["OptimizedQuantity"] * security_data[1])
        total_data = (str(i["Asset"]),float(i["OptimizedQuantity"])) + tuple(security_data)
        new_portfolio.append(total_data)
        
fields = ["Ticker","Quantity"]+fields+["Total Value"]
    
#PixieDust Visualization
sqlContext=SQLContext(sc)
d1 = sqlContext.createDataFrame(new_portfolio,fields)
display(d1)

#Debug
#print(fields)
#print(new_portfolio)

Ticker,Quantity,Name,Last Close Price,Has Military,Environmental,Total Value
FBCVX,8.52802862477,Fidelity Blue Chip Value Fund,18.76,0,High,159.985817001
FLPSX,4.19366728603,Fidelity Low-Priced Stock Fund,54.09,0,High,226.835463501
FEQTX,1.18725515567,Fidelity Equity Dividend Income Fund,27.98,0,High,33.2193992558
FGRIX,29.5366532037,Fidelity Growth & Income Portfolio Fund,35.15,0,Low,1038.21336011
FLCEX,0.809691929125,Fidelity Large-Cap Core Enhanced Index Fund,13.65,0,High,11.0522948326
FLCSX,32.8861083474,Fidelity Large-Cap Stock Fund,31.39,0,Low,1032.29494103
FFIDX,10.8622383273,Fidelity Fund,46.25,0,Low,502.378522639
FDSVX,8.15067992285,Fidelity Growth Discovery Fund,31.76,0,Average,258.86559435
FDGRX,13.0185175514,Fidelity Growth Company Fund,171.49,0,High,2232.54557489
FCPGX,2.7714967257,Fidelity Small-Cap Growth Fund,24.06,0,Low,66.6822112203


## How close did we match the risk of the benchmark?

In [62]:
print("The difference in variance between the benchmark and the optimized portfolio is: ")
print("%.8f" % json.loads(r.text)["Metadata"]["ObjectiveValue"])

The difference in variance between the benchmark and the optimized portfolio is: 
0.00000165


## Checking that the constraints are met:

In [49]:
#Sum up quantity * price
portfolio_value = 0
for j in new_portfolio:
    portfolio_value += j[-1]
print("Total Portfolio Value: " + str(portfolio_value))

#We used a Spark Data frame to display the results nicely above. Now we convert it back to a pandas dataframe to do our aggregation analysis:
OptimizedPortfolio = d1.toPandas()
for f in fields[4:-1]:
    print("Aggregation: " + f)
    elements = set(OptimizedPortfolio[f])
    for e in elements:
        aggr = OptimizedPortfolio.filter(items=fields)[OptimizedPortfolio[f] == e]
        aggr_pct = aggr["Total Value"].sum() / portfolio_value
        print("  Total allocation to " + str(e) + " is " + str(aggr_pct*100) + "%.")

Total Portfolio Value: 42118.1084206
Aggregation: Has Military
  Total allocation to 0 is 100.0%.
Aggregation: Environmental
  Total allocation to High is 49.9773398728%.
  Total allocation to Average is 10.4686954441%.
  Total allocation to Low is 39.5539646831%.
